# Event-Detecting Audio Recorder using Waggle

In [1]:
from waggle.data.audio import Microphone
from waggle.data.audio import AudioFolder
from collections import deque
import matplotlib.pyplot as plt
import ipywidgets as widgets
import numpy as np
import soundfile
import time
from datetime import datetime
import os

using backwards compatible implementation of time_ns


In [37]:
# record_events() # # # # # # #
#
# sr: sample rate
# threshold: energy cutoff point
# record_len: amount of frames to record
# frame_len: length of a frame in seconds
# buffer_len: length of buffer in frames
# path: path to write wav files to
# verbose: level of output to write
#
# # # # # # # # # # # # # # # #
def record_events(sr, threshold, frame_len, buffer_len, record_len, path, verbose):
    # establish variables
    job_list = deque()
    job_flags = deque()
    buffer = []
    record_elapse = 0
    save_count = 0
    early_event = -1
    late_event = -1
    wave_len = (buffer_len * 2) + 1 # length in frames of full recording
    event_frames = [] # list of frames that were determined to be events
    
    # initialize microphone
    microphone = Microphone()
    
    # clear directory
    delete_events(path)
    
    # Begin record loop
    print_info(1, verbose, "* Beginning recording cycle...")
    while record_elapse < record_len:
    
        # Record frame and append to end of buffer
        frame = microphone.record(frame_len)
        buffer.append(frame)
        print_info(2, verbose, f"\n** Frame {record_elapse + 1}")
        
        # if there are jobs, handle them
        if len(job_list):
            for cnt, flag in enumerate(job_flags):
                job_flags[cnt] = flag - 1
            
            # Save and delete completed jobs
            if job_flags[0] == 0:
                wave = np.append(frames_to_wave(job_list[0]), frames_to_wave(buffer[-(buffer_len):]))
                soundfile.write(path + f"{datetime.now()}.wav", wave, sr)
                print_info(3, verbose, f"*** Saved event {save_count} (length: {len(job_list[0])} frames) on frame {record_elapse}, jobs remaining: {len(job_list)}")
                save_count += 1
                job_list.popleft()
                job_flags.popleft()
                
        # Check if frame passes threshold
        if np.max(frame.data) > threshold:
            # mark frame as an event
            event_frames.append(record_elapse)
            
            # if we haven't filled the buffer yet, note when the early event was detected
            if record_elapse < buffer_len and early_event == -1:
                early_event = record_elapse
                job_list.append(buffer[:])
                job_flags.append(buffer_len)
                print_info(2, verbose, f"** Event detected on frame {record_elapse} (early)")
            # if we've reached the final buffer, note when the late event was detected
            elif record_elapse >= record_len - buffer_len and late_event == -1:
                late_event = record_len - record_elapse
                print_info(2, verbose, f"** Event detected on frame {record_elapse} (late)")
            # if event isn't early or late, then add as a job
            else:
                job_list.append(buffer[-(buffer_len):])
                job_flags.append(buffer_len)
                print_info(2, verbose, f"** Event detected on frame {record_elapse}")
            
        # increment loop counter
        record_elapse += 1
        
    # If there is a late event, save it
    if late_event != -1:
        wave = frames_to_wave(buffer[-(2*buffer_len):])
        soundfile.write(path + f"{datetime.now()}.wav", wave, sr)
        print_info(3, verbose, f"*** Saved event {save_count} on frame {record_elapse}, jobs remaining: {len(job_list)}")
        save_count += 1
    
    # when finished, print statistics
    print_info(1, verbose, f"\n* Done recording. Events saved: {save_count}")
    
    # run analysis
    full_wave = frames_to_wave(buffer)
    time = np.arange(0, len(full_wave) / sr, 1/sr)
    frame_in_samples = sr * frame_len
    
    # full buffer analysis
    plt.figure(figsize=(17, 5))
    plt.axes(ybound=(-1, 1))
    plt.title(f"Full Recording")
    plt.xlabel("Time [s]")
    plt.plot(time, full_wave)
    plt.axhline(0, color='k')
    plt.axhline(threshold, color='r')
    plt.grid(axis='x', markevery=buffer_len)
    plt.tight_layout()
    plt.show()
    
    # event analysis
    dataset = AudioFolder(path)
        
    for cnt, sample in enumerate(dataset):
        time = np.arange(0, len(sample.data) / sample.samplerate, 1/sample.samplerate)
    
        plt.figure(cnt, figsize=(17, 5))
        plt.axes(ybound=(-1, 1))
        plt.title(f"Event {cnt + 1}")
        plt.xlabel("Time [s]")
        plt.plot(time, sample.data) # plot full event
        plt.axhline(0, color='k')
        plt.axhline(threshold, color='r') # plot threshold line
        plt.grid(axis='x', markevery=buffer_len)
        plt.tight_layout()
        plt.show()

In [3]:
def frames_to_wave(buffer):
    wave = np.array([])
    for i in buffer:
        wave = np.append(wave, i.data)
        
    return wave

In [21]:
def print_info(mode, verbose, message):
    if mode <= int(verbose):
        print(message)

In [6]:
def delete_events(path):
    for x in os.listdir(path):
        if x[-4:] == ".wav":
            os.unlink(f"{path}/{x}")

In [38]:
interact_ui = widgets.interact_manual(record_events,
            sr = widgets.BoundedIntText(value=48000, min=0, max=100000, step=100, description='Sample rate', indent=10),
            threshold = widgets.FloatSlider(value=0.2, min=0, max=1.0, step=0.01, description='Threshold',
                        continuous_update=False, readout_format='.2f'),
            frame_len = widgets.BoundedIntText(value=5, min=1, max=7200, step=1, description='Frame length'),
            buffer_len = widgets.BoundedIntText(value=1, min=0, max=100, step=1, description='Buffer size'),
            record_len = widgets.BoundedIntText(value=5, min=0, step=1, description='Record size'),
            path = widgets.Text(value='audio_files/', description='Folder path:'),
            verbose = widgets.Dropdown(options=['0', '1', '2', '3'], value='1', description='Verbosity:'))

interactive(children=(BoundedIntText(value=48000, description='Sample rate', max=100000, step=100), FloatSlide…